In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from transformers import TFAutoModel, AutoTokenizer
import tensorflow as tf
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [3]:
df1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/train.xlsx')


In [4]:
df = df1.sample(frac=0.1, random_state=42)

In [5]:
df.head()

,Unnamed: 0,user,text
4148,2693,Cristiano,Eyes are the mirror of the soul pic.twitter.c...
7841,4600,katyperry,Bubble Butt by Major Lazer is my fav song righ...
18898,25058,NBA,The best of Tyreke Evans' 27 point performance...
4194,339,Cristiano,Thank you. @miguel_hrndez I'm sure U R going t...
8409,5646,premierleague,MINUTES TO GO\n\nThe atmosphere is building.....


In [6]:
df.shape

(2600, 3)

In [7]:
def pairs(dataset):
    tweet_pairs = []
    authors = dataset['user'].unique()

    dic = {author: set() for author in authors}

    total_pairs = len(dataset) // 2
    for author in authors:

        tweets_user = dataset[dataset['user'] == author]['text'].tolist()
        same_author_pairs = min(len(tweets_user) * (len(tweets_user) - 1) // 2, total_pairs // 2)

        sampled_indices_same = np.random.choice(len(tweets_user), size=(same_author_pairs, 2))
        for i1, i2 in sampled_indices_same:


            if (i1 not in dic[author]) and (i2 not in dic[author]):

                tweet_pairs.append((tweets_user[i1], tweets_user[i2], author, author, 1))
                dic[author].add(i1)
                dic[author].add(i2)
                total_pairs -= 1

        diff_author_pairs = total_pairs // (len(authors) - 1)
        for other_author in authors:
            if other_author == author:
                continue
            tweets_by_other_author = dataset[dataset['user'] == other_author]['text'].tolist()

            num_pairs_with_other_author = min(len(tweets_user) * len(tweets_by_other_author), diff_author_pairs)


            sampled_indices_diff = np.random.choice(len(tweets_by_other_author), size=(num_pairs_with_other_author, 2))
            for i1, i2 in sampled_indices_diff:
                if (i1 not in dic[author]) and (i2 not in dic[other_author]):

                    tweet_pairs.append((tweets_user[np.random.randint(len(tweets_user))], tweets_by_other_author[i1], author, other_author, 0))
                    dic[author].add(i1)


                    dic[other_author].add(i2)
                    total_pairs -= 1

        if total_pairs <= 0:
            break

    df = pd.DataFrame(tweet_pairs, columns=['text1', 'text2', 'u1', 'u2', 'isSimilar'])

    return df

NewDf =  pairs(df)


In [8]:
NewDf.head()

,text1,text2,u1,u2,isSimilar
0,#ThrowbackThursday to my launch event in Madri...,I love to hear this girl sing. She's only 13! ...,Cristiano,Cristiano,1
1,Good choice. Share how it turns out. @h_alban ...,http://say.ly/CJP4WX - Thank you for helping m...,Cristiano,Cristiano,1
2,Get out and play games with the @nikefootball ...,Summer for sure. @veronyka86 @Cristiano Hi wha...,Cristiano,Cristiano,1
3,Hey everyone! What do you think of me followin...,My CR7 keyboard got more than 500.000 download...,Cristiano,Cristiano,1
4,I hope you enjoy my new @Cr7Footwear FW16 Coll...,Mood pic.twitter.com/mjoSdnt7OE,Cristiano,Cristiano,1


In [9]:
pip install transformers


In [10]:
pip install torch transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.8 MB/s eta 0:00:00


In [11]:
# prompt: do the train test
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
transformer_model = TFAutoModel.from_pretrained(model_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(NewDf[['text1', 'text2']], NewDf['isSimilar'], test_size=0.2)


import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
import numpy as np

# Load pre-trained tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
transformer = DistilBertModel.from_pretrained("distilbert-base-uncased")

# Tokenize input data
def tokenize_data(data1, data2, max_length=128):
    inputs = tokenizer(data1, data2, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    return inputs

# Convert labels to tensor
y_train_tensor = torch.tensor(y_train.values)
y_test_tensor = torch.tensor(y_test.values)

# Tokenize train and test data
train_inputs = tokenize_data(X_train['text1'].tolist(), X_train['text2'].tolist())
test_inputs = tokenize_data(X_test['text1'].tolist(), X_test['text2'].tolist())

# Define model architecture
class SiameseModel(nn.Module):
    def __init__(self, transformer):
        super(SiameseModel, self).__init__()
        self.transformer = transformer
        self.fc = nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        logits = self.fc(pooled_output)
        return torch.sigmoid(logits)

# Initialize model
model = SiameseModel(transformer)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCELoss()

# Training loop
batch_size = 8
train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

epochs = 1
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        output = model(input_ids, attention_mask)
        loss = criterion(output.squeeze(), labels.float())
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

# Evaluation
test_dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask)
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {correct / total}")


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai